In [24]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import copy
np.random.seed(0)

# Data preprocess

In [25]:
drug8 = pd.read_csv('drug8.csv')
drug3 = pd.read_csv('drug3.csv')
drug10 = pd.read_csv('drug10.csv')
d1 = pd.read_csv('disease1.csv')
d2 = pd.read_csv('disease2.csv')
d5 = pd.read_csv('disease5.csv')

In [26]:
drug3_p = set(drug3['patid'].values)
drug10_p = set(drug10['patid'].values)
d2_p = set(d2['patid'].values)
d5_p = set(d5['patid'].values)

In [27]:
meta = pd.concat([drug8,drug3,drug10,d1,d2,d5]).drop_duplicates('patid')[['patid','GDR_CD','yrdob']]

C:\Users\peter\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [28]:
last_date = max(d1['lst_dt'])
d1 = d1.drop_duplicates('patid', keep='first')
d2 = d2.drop_duplicates('patid', keep='first')
d5 = d5.drop_duplicates('patid', keep='first')
drug8 = drug8[drug8['VXX_dt']<=last_date-60]
drug8_p = set(drug8['patid'].values)
d1_p = set(d1['patid'].values)

# Generate new records

In [29]:
patient = set(drug8['patid'].values)|\
          set(drug3['patid'].values)|\
          set(drug10['patid'].values)|\
          set(d2['patid'].values)|\
          set(d5['patid'].values)-\
          set(d1.loc[d1['fst_dt']<60,'patid'].values)

In [30]:
for i in patient:
    if i in drug8_p:
        continue

In [31]:
d1.index = d1['patid']
drug8.index = drug8['patid']

In [33]:
patient_info = {'patid':[], 
       'date' : [],
       'd1_record': [],
       'drug8_record':[]
      }
sample_date = []
for i in patient:
    if i in drug8_p: #If take the drug
        patient_info['patid'].append(i)
        if i in d1_p: #If have the disease 1
            diagnose =  d1.loc[[i],'fst_dt'].values[0]
            sub_tmp = drug8.loc[[i],:]
            sub = sub_tmp[(sub_tmp['VXX_dt']<diagnose-14) &  (sub_tmp['sup_dt']>diagnose-60)]
            if len(sub) == 0: #if no record satisfies the time condition
                patient_info['date'].append(np.random.choice(sub_tmp['VXX_dt'].values))
                patient_info['d1_record'].append(0) #randomly pick one record
            else: #if at least one record satisfies the time condition
                patient_info['date'].append(sub['VXX_dt'].values[0])
                patient_info['d1_record'].append(1) #randomly pick the first record satisfying the time condition
        else: #If no disease 1
            patient_info['date'].append(np.random.choice(drug8.loc[[i],'VXX_dt'].values))
            sample_date.append(patient_info['date'][-1])
            patient_info['d1_record'].append(0) # randomly pick one record
        patient_info['drug8_record'].append(1) 
        
for i in patient:
    if i not in drug8_p:
        patient_info['patid'].append(i)
        if i in d1_p: #If have the disease 1
            patient_info['date'].append(d1.loc[[i], 'fst_dt'].values[0] - np.random.randint(14,60))
            patient_info['d1_record'].append(1) #randomly pick a date 14-60 days between getting the disease
        else: #If no disease 1
            patient_info['date'].append(np.random.choice(sample_date))
            patient_info['d1_record'].append(0) #randomly pick a date between 500 and 2000
        patient_info['drug8_record'].append(0)
patient_info = pd.DataFrame(patient_info)    

KeyboardInterrupt: 

In [9]:
patient_info_tmp = pd.merge(patient_info, drug3, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug3']=1 #If taking drug3 before the date, set column ‘drug3’ to be 1
patient_info_tmp.loc[patient_info_tmp['drug3']!=1,'drug3']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10=patient_info_tmp_10[['patid','drug3']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug3'] == 1]
patient_info_tmp_date.loc[:,'drug3_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug3_date']<0,'drug3_date'] = 0 #exacting all record with drug3 = 1, calculate the latest time taking drug 3
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug3_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left') #merge

patient_info_tmp = pd.merge(patient_info, drug10, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug10']=1
patient_info_tmp.loc[patient_info_tmp['drug10']!=1,'drug10']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10=patient_info_tmp_10[['patid','drug10']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug10'] == 1]
patient_info_tmp_date.loc[:,'drug10_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug10_date']<0,'drug10_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug10_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug8, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug8']=1
patient_info_tmp.loc[patient_info_tmp['drug8']!=1,'drug8']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10=patient_info_tmp_10[['patid','drug8']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug8'] == 1]
patient_info_tmp_date.loc[:,'drug8_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug8_date']<0,'drug8_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug8_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d2, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d2']=1
patient_info_tmp.loc[patient_info_tmp['d2']!=1,'d2']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10=patient_info_tmp_10[['patid','d2']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d2'] == 1]
patient_info_tmp_date.loc[:,'d2_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d2_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d5, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d5']=1
patient_info_tmp.loc[patient_info_tmp['d5']!=1,'d5']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10=patient_info_tmp_10[['patid','d5']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d5'] == 1]
patient_info_tmp_date.loc[:,'d5_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d5_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info = pd.merge(patient_info,meta, on='patid', how='left')

C:\Users\peter\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\peter\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
patient_info_8 = patient_info[patient_info['drug8_record']==1]
patient_info_no_8 = patient_info[patient_info['drug8_record']==0]

In [11]:
len(red_8[red_8['d1_record']==1]), len(red_8[red_8['d1_record']==0]), len(red_no_8[red_no_8['d1_record']==1]), len(red_no_8[red_no_8['d1_record']==0])

(127, 233410, 867, 204555)

In [12]:
d = np.array([[127, 233410], [867, 204555]])
chi2_contingency(d)

(652.3070287225964,
 7.036865370774887e-144,
 1,
 array([[   528.83248322, 233008.16751678],
        [   465.16751678, 204956.83248322]]))

In [17]:
import pickle 
with open('red.pkl', 'wb') as f:
    pickle.dump(patient_info_8, f)
with open('red_no.pkl', 'wb') as f:
    pickle.dump(patient_info_no_8, f)

In [34]:
import pickle
with open('red.pkl', 'rb') as f:
    patient_info_8 = pickle.load(f)
    
with open('red_no.pkl', 'rb') as f:
    patient_info_no_8 = pickle.load(f)

In [9]:
patient_info_8['drug3'] = patient_info_8['drug3'].apply(int).apply(str)
patient_info_8['drug10'] = patient_info_8['drug10'].apply(int).apply(str)
patient_info_8['drug8'] = patient_info_8['drug8'].apply(int).apply(str)
patient_info_8['d2'] = patient_info_8['d2'].apply(int).apply(str)
patient_info_8['d5'] = patient_info_8['d5'].apply(int).apply(str)
patient_info_8['yrdob'] = patient_info_8['yrdob'].apply(int).apply(str)

In [8]:
patient_info_no_8['drug3'] = patient_info_no_8['drug3'].apply(int).apply(str)
patient_info_no_8['drug10'] = patient_info_no_8['drug10'].apply(int).apply(str)
patient_info_no_8['drug8'] = patient_info_no_8['drug8'].apply(int).apply(str)
patient_info_no_8['d2'] = patient_info_no_8['d2'].apply(int).apply(str)
patient_info_no_8['d5'] = patient_info_no_8['d5'].apply(int).apply(str)
patient_info_no_8['yrdob'] = patient_info_no_8['yrdob'].apply(int).apply(str)

In [10]:
patient_info_8.loc[:, 'key'] = patient_info_8['drug3']+patient_info_8['drug10']+patient_info_8['drug8']+patient_info_8['d2']+patient_info_8['d5']+patient_info_8['GDR_CD']+patient_info_8['yrdob']
patient_info_no_8.loc[:,'key'] = patient_info_no_8['drug3']+patient_info_no_8['drug10']+patient_info_no_8['drug8']+patient_info_no_8['d2']+patient_info_no_8['d5']+patient_info_no_8['GDR_CD']+patient_info_no_8['yrdob']

In [13]:
patient_info_8.index = np.arange(len(patient_info_8))
patient_info_no_8.index = np.arange(len(patient_info_no_8))

In [15]:
array1 = patient_info_8['key'].values
res1 = patient_info_8['d1_record'].values
array2 = patient_info_no_8['key'].values
res2 = patient_info_no_8['d1_record'].values

In [16]:
res1 = res1[np.argsort(array1)]
array1 = array1[np.argsort(array1)]
res2 = res2[np.argsort(array2)]
array2 = array2[np.argsort(array2)]

In [17]:
point1a = 0
point1b = 0
point2a = 0
point2b = 0
res1_tot = 0
res2_tot = 0 
tot = 0
while point1a < len(res1):
    while array1[point1b] == array1[point1a]:
        point1b += 1
        if point1b >= len(res1):
            break
    tmp1 = res1[point1a:point1b]
    if array2[point2a]<array1[point1a]:
        while array2[point2a]<array1[point1a]:
            point2a+=1
            if point2a >= len(res2):
                break
    if point2a >= len(res2):
        break
    if array2[point2a]>array1[point1a]:
        point1a = point1b
        point1b = point1a
        continue
    else:
        point2b = point2a
        while array2[point2b] == array2[point2a]:
            point2b += 1
        tmp2 = res2[point2a:point2b]
        tot += len(tmp1)
        res1_tot += np.sum(tmp1)
        res2_tot += np.mean(tmp2)*len(tmp1)
        point1a = point1b
        point1b = point1a
        point2a = point2b
        point2b = point2a


In [48]:
point1a = 0
point1b = 0
point2a = 0
point2b = 0
res1_tot = 0
res2_tot = 0 
tot = 0
while point1a < len(res1):
    while array1[point1b] == array1[point1a]:
        point1b += 1
        if point1b >= len(res1):
            break
    tmp1 = res1[point1a:point1b]
    if array2[point2a]<array1[point1a]:
        while array2[point2a]<array1[point1a]:
            point2a+=1
            if point2a >= len(res2):
                break
    if point2a >= len(res2):
        break
    if array2[point2a]>array1[point1a]:
        point1a = point1b
        point1b = point1a
        continue
    else:
        point2b = point2a
        while array2[point2b] == array2[point2a]:
            point2b += 1
        tmp2 = res2[point2a:point2b]
        tot += (len(tmp1)+len(tmp2))/2
        res1_tot += np.mean(tmp1)*(len(tmp1)+len(tmp2))/2
        res2_tot += np.mean(tmp2)*(len(tmp1)+len(tmp2))/2
        point1a = point1b
        point1b = point1a
        point2a = point2b
        point2b = point2a


In [49]:
tot

194368.5

In [50]:
res1_tot

67.61968395188565

In [51]:
res2_tot

876.4956429376175

# Matching with binary columns

In [90]:
res1_tot = 0
res2_tot = 0 
tot = 0
for i in np.unique(array1):
    tmp=res2[array2==i]
    if(len(tmp))>0:
        tmp1 = res1[array1==i]
        tot += len(tmp1)
        res1_tot += np.sum(res1[array1==i])
        res2_tot += np.mean(tmp)*len(tmp1)

In [14]:
red_no_8_copy = copy.deepcopy(red_no_8)
red_8.index = np.arange(len(red_8))
no_index = []
yes_index = []
for i in range(len(red_8)):
    v1 = red_8.loc[i, 'drug3']
    v2 = red_8.loc[i, 'drug10']
    v3 = red_8.loc[i, 'drug8']
    v4 = red_8.loc[i, 'd2']
    v5 = red_8.loc[i, 'd5']
    v6 = red_8.loc[i, 'GDR_CD']
    v7 = red_8.loc[i, 'yrdob']
    match = red_no_8_copy[(red_no_8_copy['drug3']==v1)&
                          (red_no_8_copy['drug10']==v2)&
                          (red_no_8_copy['drug8']==v3)&
                          (red_no_8_copy['d2']==v4)&
                         (red_no_8_copy['d5']==v5)&
                         (red_no_8_copy['GDR_CD']==v6)&
                         (red_no_8_copy['yrdob']==v7)]
    if(len(match)>0):
        yes_index.append(i)
        tmp = match.index[0]
        no_index.append(tmp)
        red_no_8_copy.drop(tmp,axis=0, inplace=True)
no_drug = red_no_8.loc[no_index,:]
yes_drug = red_8.loc[yes_index,:]

In [16]:
len(no_drug[no_drug['d1_record']==1])

410

In [17]:
len(no_drug[no_drug['d1_record']==0])

94736

In [18]:
len(yes_drug[yes_drug['d1_record'] == 1])

29

In [19]:
len(yes_drug[yes_drug['d1_record'] == 0])

95117

In [2]:
d = np.array([[410, 94736], [29, 95117]])
chi2_contingency(d)

(329.68997342111004, 1.1228239172459087e-73, 1, array([[  219.5, 94926.5],
        [  219.5, 94926.5]]))

# Matching with date columns

In [17]:
red_no_8_copy = copy.deepcopy(red_no_8)
red_8.index = np.arange(len(red_8))
red_no_8_copy = copy.deepcopy(red_no_8)
no_index = []
yes_index = []
for i in range(len(red_8)):
    match = red_no_8_copy
    v1 = red_8.loc[i, 'drug3_date']
    v2 = red_8.loc[i, 'drug10_date']
    v3 = red_8.loc[i, 'drug8_date']
    v4 = red_8.loc[i, 'd2_date']
    v5 = red_8.loc[i, 'd5_date']
    
    if np.isnan(v1):
        match = match[match['drug3_date'].isna()]
    else:
        match = match[match['drug3_date'] == v1]
    if np.isnan(v2):
        match = match[match['drug10_date'].isna()]
    else:
        match = match[match['drug10_date'] == v2]
    if np.isnan(v3):
        match = match[match['drug8_date'].isna()]
    else:
        match = match[match['drug8_date'] == v3]
    if np.isnan(v4):
        match = match[match['d2_date'].isna()]
    else:
        match = match[match['d2_date'] == v4]
    if np.isnan(v5):
        match = match[match['d5_date'].isna()]
    else:
        match = match[match['d5_date'] == v5]

    v6 = red_8.loc[i, 'GDR_CD']
    v7 = red_8.loc[i, 'yrdob']
    match = match[(match['GDR_CD']==v6) & (match['yrdob']==v7)]
    if(len(match)>0):
        yes_index.append(i)
        tmp = match.index[0]
        no_index.append(tmp)
        red_no_8_copy.drop(tmp,axis=0, inplace=True)
no_drug = red_no_8.loc[no_index,:]
yes_drug = red_8.loc[yes_index,:]

In [19]:
len(no_drug[no_drug['d1_record']==1])

399

In [20]:
len(no_drug[no_drug['d1_record']==0])

91400

In [21]:
len(yes_drug[yes_drug['d1_record'] == 1])

28

In [22]:
len(yes_drug[yes_drug['d1_record'] == 0])

91771

In [23]:
d = np.array([[408, 91178], [28, 91558]])
chi2_contingency(d)

(330.237892344217, 8.530493248095933e-74, 1, array([[  218., 91368.],
        [  218., 91368.]]))

In [36]:
patient_info_8 = patient_info_8.drop(['drug3_date','drug10_date','drug8_date','d2_date','d5_date'], axis=1)
patient_info_no_8 = patient_info_no_8.drop(['drug3_date','drug10_date','drug8_date','d2_date','d5_date'], axis=1)

In [37]:
from pymatch.Matcher import Matcher

features_excluded = ['patid','date','d1_record','drug10']
matcher = Matcher(patient_info_8, patient_info_no_8, yvar='drug8_record', exclude=features_excluded)

Formula:
drug8_record ~ drug3+drug8+d2+d5+GDR_CD+yrdob
n majority: 233537
n minority: 205422


In [42]:
patient_info_8['drug8_record']

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
233507    1
233508    1
233509    1
233510    1
233511    1
233512    1
233513    1
233514    1
233515    1
233516    1
233517    1
233518    1
233519    1
233520    1
233521    1
233522    1
233523    1
233524    1
233525    1
233526    1
233527    1
233528    1
233529    1
233530    1
233531    1
233532    1
233533    1
233534    1
233535    1
233536    1
Name: drug8_record, Length: 233537, dtype: int64

In [38]:
matcher.fit_scores(balance=False, nmodels=1)

Fitting 1 (Unbalanced) Model...


ValueError: Unable to coerce to Series, length must be 1: given 438959

In [40]:
matcher

In [43]:
import sqlite3